# An Introduction to Retrieval Augmented Generation (RAG)

## Just a quick definition
Retrieval Augmented Generation (RAG) is a technique that combines information retrieval with generative models, allowing the system to fetch relevant documents from a knowledge base and use them to generate more accurate and informed responses.

![OpenAI RAG Definition](../../images/openai-rag-image.png)

*Image courtesy of [OpenAI](https://help.openai.com/en/articles/8868588-retrieval-augmented-generation-rag-and-semantic-search-for-gpts)*

In [1]:
import os
import glob
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    raise Exception("API key is missing")

In [2]:
MODEL = "gpt-4o-mini"

In [3]:
context = {}

employees = glob.glob("docs/*")

for employee in employees:
    name = os.path.basename(employee).split()[0]
    with open(employee, "r") as file:
        content = file.read()
    context[name] = content

In [4]:
context

{'David': 'STERLING & ASSOCIATES LAW GROUP - ATTORNEY RECORD\n===============================================\n\nAttorney ID: SAL-035\nName: David Michael Thompson\nPosition: Partner - Employment Law & Labor Relations\nDepartment: Employment & Labor\nBar Admission: New York (2008), New Jersey (2009), Connecticut (2010)\nHire Date: June 1, 2008\n\nPROFESSIONAL BACKGROUND:\n------------------------\n\nCurrent Position (2018 - Present):\nPartner - Employment Law & Labor Relations\n- Leads firm\'s employment law and labor relations practice\n- Represents management in complex employment disputes\n- Handles class action employment litigation defense\n- Manages team of 8 attorneys and 4 paralegals\n- Annual billable target: 1,950 hours (averages 2,200 hours)\n\nPrevious Position (2013 - 2018):\nSenior Associate - Employment Law\n- Specialized in employment litigation and compliance\n- Developed expertise in wage and hour disputes\n- Built reputation for successful trial outcomes\n- Led major

In [5]:
# build a function which will insspect a tect prompt for names of employees
def get_context(prompt):
    names = []
    for name in context.keys():
        if name.lower() in prompt.lower():
            return context[name]

In [6]:
prompt = "I want to know about Sarahs role in the project."

get_context(prompt)

'STERLING & ASSOCIATES LAW GROUP - ATTORNEY RECORD\n===============================================\n\nAttorney ID: SAL-027\nName: Sarah Michelle Chen\nPosition: Senior Associate - Intellectual Property & Technology Law\nDepartment: Intellectual Property & Technology\nBar Admission: New York (2018), Patent Bar (USPTO, 2019)\nHire Date: September 10, 2018\n\nPROFESSIONAL BACKGROUND:\n------------------------\n\nCurrent Position (2022 - Present):\nSenior Associate - IP & Technology Law\n- Lead attorney for intellectual property portfolio management\n- Handles patent prosecution, trademark disputes, and licensing agreements\n- Specializes in technology transfer and software licensing\n- Manages relationships with 25+ technology clients\n- Annual billable target: 2,100 hours (consistently exceeds)\n\nPrevious Position (2018 - 2022):\nAssociate - IP & Technology Law\n- Started in patent prosecution and IP due diligence\n- Developed expertise in DMCA compliance and privacy law\n- Built pract

In [7]:
def add_context(prompt):
    context = get_context(prompt)
    if context:
        prompt += "\n\nThe following information is relevant:\n\n" + context
    return prompt

In [8]:
print(add_context(prompt))

I want to know about Sarahs role in the project.

The following information is relevant:

STERLING & ASSOCIATES LAW GROUP - ATTORNEY RECORD

Attorney ID: SAL-027
Name: Sarah Michelle Chen
Position: Senior Associate - Intellectual Property & Technology Law
Department: Intellectual Property & Technology
Bar Admission: New York (2018), Patent Bar (USPTO, 2019)
Hire Date: September 10, 2018

PROFESSIONAL BACKGROUND:
------------------------

Current Position (2022 - Present):
Senior Associate - IP & Technology Law
- Lead attorney for intellectual property portfolio management
- Handles patent prosecution, trademark disputes, and licensing agreements
- Specializes in technology transfer and software licensing
- Manages relationships with 25+ technology clients
- Annual billable target: 2,100 hours (consistently exceeds)

Previous Position (2018 - 2022):
Associate - IP & Technology Law
- Started in patent prosecution and IP due diligence
- Developed expertise in DMCA compliance and privacy l

In [ ]:
system_message = """

You are a helpful assistant. You will answer questions based only on the provided context. 

If you do not have information, do not make anything up.

"""

In [21]:
client = OpenAI()

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history
    message = add_context(message)
    messages.append({"role": "user", "content": message})

    stream = client.responses.create(
        model=MODEL,
        input=messages,
        stream=True
    )

    for event in stream:
        if event.type == "response.output_text.delta":
            print(event.delta, end="")

In [23]:
display(Markdown(chat("What is Sarah's role in the project?", [])))

Sarah Michelle Chen serves as a Senior Associate in the Intellectual Property & Technology Law department at Sterling & Associates Law Group. Her primary roles in the project include:

- **Lead Attorney for Intellectual Property Portfolio Management**: Overseeing and managing clients' IP portfolios.
- **Handling Patent Prosecution and Trademark Disputes**: Actively engaged in filing and defending patents and trademarks.
- **Specializing in Technology Transfer and Software Licensing**: Focusing on the legal aspects of technology transfer and agreements related to software.
- **Managing Client Relationships**: Overseeing relationships with over 25 technology clients, ensuring their legal needs are met.

Her extensive background in both law and technology, alongside her notable achievements and technical expertise, positions her as a key player in navigating complex IP legal matters within the project.

<IPython.core.display.Markdown object>

In [29]:
client = OpenAI()

def chat(message, history):
    # Convert Gradio history format to OpenAI messages format
    messages = [{"role": "system", "content": system_message}]
    
    # Process history - Gradio passes it as list of [user_msg, assistant_msg] pairs
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    
    # Add context to current message and append
    message = add_context(message)
    messages.append({"role": "user", "content": message})

    stream = client.responses.create(
        model=MODEL,
        input=messages,
        stream=True
    )

    accumulated_content = ""
    
    for event in stream:
        if event.type == "response.output_text.delta":
            accumulated_content += event.delta
            yield accumulated_content

In [32]:
view = gr.ChatInterface(chat).launch()

/home/chucky/projects/ai-engineering/ai_env/lib/python3.12/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
